In [1]:
import mysql.connector
#bpcircuits env

# Idea
Build an application where people can log what circuit color they have done each day. Afterwards, show statistics for data collected for each user. <br>
Learn how to properlly store data in a database (mongo? SQL?) as well as username/password authentication.

To start, just make something where users can log their sends by color. Yellow, Red, Green, Purple, Orange, Black, Blue, Pink, White, Wood/Other.

In [1]:
import sqlite3
from datetime import datetime
import time

def create_connection():
    try:
        conn = sqlite3.connect('climbing_log.db')
        return conn
    except sqlite3.Error as e:
        print(f'Error creating connection to database: {e}')
        return None

def setup_database():
    conn = create_connection()  # Call the function correctly
    if conn is not None:
        cursor = conn.cursor()
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT NOT NULL UNIQUE,
            password TEXT NOT NULL,
            email TEXT NOT NULL
        )
        ''')
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS climbs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            color TEXT,
            date TEXT,
            location TEXT,
            FOREIGN KEY(user_id) REFERENCES users(id)
        )
        ''')
        
        conn.commit()
        conn.close()
    else:
        print('Faild to create a connection to the database.')

def clear_db():
    conn = create_connection()
    if conn is not None:
        cursor = conn.cursor()
        cursor.execute('DROP TABLE IF EXISTS users')
        cursor.execute('DROP TABLE IF EXISTS climbs')
        conn.commit()
        conn.close()
        print('Database has been cleared')
    else:
        print('Failed to connect to the database')


In [11]:
class User:
    def __init__(self, username, password, email):
        self.username = username
        self.password = password
        self.email = email
        self.user_id = None
        #self.log_climb = {}     #dictionary for climbs, key:date OUTDATED
        #self.session_notes = {} #ability to make notes about the session, not specific climbs OUTDATED

    def save_to_db(self):
        conn = create_connection()
        if conn is not None:
            cursor = conn.cursor()
            try:
                cursor.execute('INSERT INTO users (username, password, email) VALUES (?, ?, ?)',
                            (self.username, self.password, self.email))
                conn.commit()
                self.user_id = cursor.lastrowid
                print(f"User saved with user_id: {self.user_id}")  # Debug print
            except sqlite3.IntegrityError as e:
                print(f'Integrity error: {e}')
            finally:
                conn.close()
        else:
            print('Failed to create the database connection.')
            
    @staticmethod
    def get_user(username):
            conn = create_connection()
            if conn is not None:
                cursor = conn.cursor()
                cursor.execute('SELECT * FROM users WHERE username = ?', (username,))
                user_data = cursor.fetchone()
                conn.close()
                
                if user_data:
                    user = User(user_data[1], user_data[2], user_data[3])
                    user.user_id = user_data[0]
                    return user
                else:
                    return None
            else:
                print('Failed to create the database connection.')
                return None
        
    def log_climb(self, color, date, location= 'BP Minneapolis'):
        conn = create_connection()
        if conn is not None:
            cursor = conn.cursor()
            try:
                cursor.execute('INSERT INTO climbs (user_id, color, date, location) VALUES (?, ?, ?, ?)',
                            (self.user_id, color, date, location))
                conn.commit()
            except sqlite3.OperationalError as e:
                print(f'Operational error: {e}')
                if str(e) == 'database is locked':
                    #retry
                    retry_count = 5
                    for i in range(retry_count):
                        time.sleep(1)
                        try:
                            cursor.execute('INSERT INTO climbs (user_id, color, date, location) VALUES (?, ?, ?, ?)',
                                           (self.user_id, color, date, location))
                            conn.commit()
                            break
                        except sqlite3.OperationalError as e:
                            if i == retry_count - 1:
                                print("Max retries reached. Could not log climb.")
            finally:
                conn.close()
        else:
            print('Failed to create the database connection.')

    def create_user(username, password, email):
        user = User(username, password, email)
        user.save_to_db()
        if user.user_id is not None:
            print(f'User {username} sucessfully created with user_id: {user.user_id}')
        else:
            print(f'Failed to create new user {username}. This name may already exist.')

class Boulder:
    def __init__(self, color, date, location):
        self.color = color
        self.date = date
        self.location = location


In [13]:
	# Creating a user using the User.create_user() function
#clear_db()  #clearing any other users for testing purposes. Will have to change when I want to test the ability to handle duplicate usernames.
setup_database()
User.create_user('sample1', '12345', 'test@email.com')

sample_user = User.get_user('sample1')
print(f'Sample user ID {sample_user.user_id}')

#Can we save climbs now?
# Ensure the user was saved correctly and can log climbs
#       Current issue: Even if a new user cannot be created, the command of logging a climb still happens.
#       This is why the 'and 0 < 1' part exists. I haven't found a way to check if the action of creating a new user has happened. Will probably have to
#       Add a new loop that can break if it returns that a user already exists.

if sample_user.user_id is not None and 0 < 1:
    sample_user.log_climb('Purple', '2024-05-29', 'BP Minneapolis')
else:
    print("User ID is None, skipping log_climb steps.")  # Debug print


retrieved_user = User.get_user('sample1')

if retrieved_user:
    print(f'User {retrieved_user.username} found with ID {retrieved_user.user_id}')

    conn = create_connection()
    if conn is not None:
        try:
            cursor = conn.cursor()
            cursor.execute('SELECT * FROM climbs WHERE user_id = ? AND date = ?', (retrieved_user.user_id, '2024-05-29'))
            climbs = cursor.fetchall()
        except sqlite3.OperationalError as e:
            print(f'OperationalError: {e}')
        finally:
            conn.close()

        print(f'Climbs on 2024-05-29: {len(climbs)}')
        for climb in climbs:
            print(f'Climb ID: {climb[0]}, Color: {climb[2]}, Date: {climb[3]}, Location: {climb[4]}')
    else:
        print('Failed to create the database connection.')
else:
    print('User not found')

Integrity error: UNIQUE constraint failed: users.username
Failed to create new user sample1. This name may already exist.
Sample user ID 2
User sample1 found with ID 2
Climbs on 2024-05-29: 1
Climb ID: 5, Color: Purple, Date: 2024-05-29, Location: BP Minneapolis


In [11]:

    #Testing the database's ability to log climbs. Currently running into an error where climbs don't save/retrieve properly.

clear_db()

#Testing the code
setup_database()
sample_user = User('sample_user', '12345', 'sample_user@example.com')
sample_user.save_to_db()

#Checking if user_id is set up properly
print(f'Sample User ID after save_to_db: {sample_user.user_id}')
if sample_user.user_id is not None:
    sample_user.log_climb('Purple', '2024-05-29', 'BP Minneapolis')
    sample_user.log_climb('Purple', '2024-05-29', 'BP Minneapolis')
    sample_user.log_climb('Purple', '2024-05-29', 'BP Minneapolis')
else:
    print('User ID is None, skipping log_climb step.')

retrieved_user = User.get_user('sample_user')

if retrieved_user:
    print(f'User {retrieved_user.username} found with ID {retrieved_user.user_id}')

    conn = create_connection()
    if conn is not None:
        cursor = conn.cursor()
        try:
            cursor.execute('SELECT * FROM climbs WHERE user_id= ? AND date = ?', (retrieved_user.user_id, '2024-04-29'))
            climbs = cursor.fetchall()
        except sqlite3.OperationalError as e:
            print(f'OperationalError: {e}')
        finally:
            conn.close()

        print(f'Climbs on 2024-05-29: {len(climbs)}')
        for climb in climbs:
            print(f'Climb ID: {climb[0]}, Color: {climb[1]}, Date: {climb[2]}, Location: {climb[3]}')
    else:
        print('Could not create database connection')
else:
    print('User not found')



Database has been cleared
User saved with user_id: 1
Sample User ID after save_to_db: 1
User sample_user found with ID 1
Climbs on 2024-05-29: 0


In [13]:
climbs

[]